# Predicting the Dyssynchrony Index
We will be tackling a sequence classification problem with recurrent neural networks. We believe that the vectorcardiogram is a good predictor of the dyssynchrony index, so we will treat the VCG as a sequence of coordinates and feed it into a LSTM network.

## Data Dimensions

### *Input*
Our input will be (simulated) vectorcardiograms generated by Chris Villongco using CMRG's Continuity.
Our dataset consists of 608 simulated VCGs. These simulations have varying parameters, such as differing stimulus sites and conduction velocities, but are based on the same patient (patient 2, BiV2). We limit our dataset to only 608 examples in the interest of speeding up computational time, as we are more interested in showing proof of concept than obtaining a higher accuracy (of course, we will aim for higher accuracy later).

Because we view the VCG as a sequence, each timestep can be viewed as one element in the sequence. For every time ```t```, we are given three inputs, the  ```(x, y, z)``` coordinates of the vector head.

### *Output*
We will be classifying each VCG based on the corresponding dyssynchrony index from that same simulation. The dyssynchrony index theoretically ranges from 0 to 1, but we will only be concerned with the range of 0.5 to 1. This range will be further divided into 5 regular intervals. Specifically, these intervals will be ```[0.5, 0.6), [0.6, 0.7), [0.7, 0.8), [0.8, 0.9)```, and ```[0.9, 1.0]```. Simulations with dyssynchrony indices under 0.5 will be considered faulty and will be placed in the first interval, ```[0.5, 0.6)```. Also, note that simulations with dyssynchrony indices of 1.0 will be placed in the last interval, ```[0.9, 1.0]```. These intervals will be labeled as follows (intervals are 0-indexed):
* ```0: [0.5, 0.6)```
* ```1: [0.6, 0.7)```
* ```2: [0.7, 0.8)```
* ```3: [0.8, 0.9)```
* ```4: [0.9, 1.0]```
  
Thus, for example, a VCG sequence with a corresponding dyssynchrony index of 0.78 will be placed in class ```"2"``` since it falls in the range of 0.7 and 0.8, the third interval.

## Dataset Wrapper
We've created a class that provides a basic interface for handling the dataset. Specifically, the wrapper will do the following: 
* Read in the dataset from three specified ```.npy``` files (VCG, VCG lengths, target class)
* Split the dataset into training, validation, and testing sets (the set sizes will be fixed for convenience)
* Provide a ```next_batch``` function that will return a batch of specified size for a given set.

We import the wrapper here. To instantiate, we specify the names of the NumPy files for the following:
* ```vcg.npy```: VCG sequences (input)
* ```vcg_length.npy```: VCG sequence lengths (passed as argument for ```sequence_length.npy``` parameter)
* ```target.npy```: dyssynchrony indices (target output)

In [1]:
from dataset import Patient

# Initialize dataset iterator
patient_dataset = Patient("vcg.npy", "vcg_length.npy", "target.npy")

## Network Dimensions
We will define the dimensions of our data, as well as the initial hyperparameters of our neural network here. Note: these parameters have not been optimized, they are simply for proof of concept.

In [2]:
# Hyperparameters
learning_rate = 0.05
training_iters = 125
batch_size = 20
display_step = 10
num_hidden = 100

# Network Parameters
num_steps = 120
num_inputs = 3
num_classes = 5

# Where TensorFlow saves metadata for TensorBoard
logs_path='Data/'

## Input Placeholders
We define three placeholders. They are for the following:
* VCG sequence input
* VCG sequence length
* targets (used for training)

In [3]:
import tensorflow as tf 

# VCG input 
inputs = tf.placeholder(tf.float32, [None, num_inputs, num_steps])

# VCG sequence lengths
sequence_length = tf.placeholder(tf.int32, [None])

# Index of class the VCG should be categorized as
y = tf.placeholder(tf.float32, [None, num_classes])

## Weights and Biases
The recurrent neural network creates an output at every timestep. Since this is a problem of sequence classification, we are only interested in the output produced at the last timestep, ```t=t_end```. We then apply a linear activation on it. The weights and biases are initialized with random values from a normal distribution, with a mean of 0.0 and a standard deviation of 1.0.

In [4]:
# Define weights and biases
weights = tf.Variable(tf.random_normal([num_hidden, num_classes]))
biases = tf.Variable(tf.random_normal([num_classes]))

## Recurrent Neural Network Cell
Here we define what kind of recurrent neural network we will be using. We will be using a basic LSTM network with a default forget bias of 1.0, and ```tanh``` as the activation function. The ```BasicLSTMCell``` initializer function takes as parameters:
* ```num_units```: The number of units in a LSTM cell.
* ``` forget_bias```: float, the bias added to the forget gates.
* ``` activation```: activation function of the inner states. Default is ```tanh```.
* ``` state_is_tuple```: Accepted and returned states are 2-tuples of the c_state and m_state(???). Default is True

In [5]:
from tensorflow.python.ops import rnn_cell

# Define a lstm cell with tensorflow
cell = rnn_cell.BasicLSTMCell(num_units=num_hidden, forget_bias=1.0)

We will be using the ```tf.nn.dynamic_rnn``` function, instead of the ```tf.nn.rnn``` function, to get the output of the recurrent neural network. Unlike ```tf.nn.rnn```, ```tf.nn.dynamic_rnn``` takes in variable sequence lengths (it uses a ``tf.While`` loop to dynamically construct the computational graph). Also, it is faster (supposedly), despite the fact that ```tf.nn.rnn``` prebuilds the graph. The parameters are as follows:
* ```cell```: an instance of RNN cell.
* ```inputs```: the RNN input, a single Tensor. The dimensions are [batch_size, sequence_length, num_inputs]
* ```sequence_length```: (optional) An int32/64 vector of size [batch_size] specifying the length of each sequence.
* ```dtype```: (optional) The data type for the initial state and the expected output. 

In [6]:
output, states = tf.nn.dynamic_rnn(
    cell=cell,
    dtype=tf.float32,
    sequence_length=sequence_length,
    inputs=inputs
)

print "hello"

hello


TODO specify sequence lengths